In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\ASUS\\Desktop\\Mediicoss\\research'

In [2]:
import os
os.chdir("../")

In [3]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
def load_single_pdf(data):
    loader = DirectoryLoader(
        data, 
        glob="*.pdf", 
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents


In [5]:
extracted_data = load_single_pdf("data")

In [6]:
len(extracted_data)

637

In [7]:
from typing import List 
from langchain.schema import Document

def filter_to_minimal_docs(docs: list[Document]) -> list[Document]:
    minimal_docs:List[Document] = []
    for doc in docs:
        src=doc.metadata.get("source","")
        minimal_docs.append(
            Document(
            page_content=doc.page_content,
            metadata={"source":src}
        ))
    return minimal_docs

In [8]:
minimal_docs =filter_to_minimal_docs(extracted_data)

In [9]:
#Chunking
def text_split(minimal_docs):
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
        
    )
    texts_chunk=  text_splitter.split_documents(minimal_docs)
    return texts_chunk


In [10]:
test_chunk =text_split(minimal_docs)
print(f" numberof chunks:{len(test_chunk)}")










 numberof chunks:5859


In [11]:
#embedding
from langchain.embeddings import HuggingFaceEmbeddings
def Download_embeddings():
    model_name="sentence-transformers/all-MiniLM-L6-v2"
    embedding=HuggingFaceEmbeddings(
        model_name=model_name,
        
    )
    return embedding

embeddings=Download_embeddings()







C:\Users\ASUS\AppData\Local\Temp\ipykernel_3904\2920820361.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding=HuggingFaceEmbeddings(
c:\Users\ASUS\Desktop\Mediicoss\medical\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
vector=embeddings.embed_query("Hello, how are you?")
print(vector)
print(len(vector))



[0.019096743315458298, 0.034465156495571136, 0.09162802249193192, 0.07016527652740479, -0.029946595430374146, -0.08419135212898254, 0.045813582837581635, 0.004958535544574261, -0.09189330786466599, 0.017400633543729782, -0.008816155605018139, -0.0006614617886953056, -0.02855694852769375, -0.021949714049696922, 0.055166687816381454, -0.04983649402856827, 0.08988092094659805, -0.08895715326070786, -0.11235621571540833, 0.03900052234530449, -0.06607077270746231, 0.026095140725374222, 0.036530692130327225, 0.06139037385582924, -0.05712488666176796, -0.05463936924934387, 0.03036557510495186, 0.03238755092024803, 0.012644711881875992, -0.1056857779622078, -0.05834547430276871, 0.06732934713363647, -0.040755920112133026, 0.006439770571887493, 0.005698689259588718, 0.052853215485811234, -0.039775311946868896, -0.11855249851942062, 0.0021162095945328474, -0.016692854464054108, 0.028338085860013962, -0.03743797540664673, -0.021371396258473396, -0.04147520288825035, 0.08497177809476852, -0.068694

In [14]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [15]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [16]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY
pc = Pinecone(api_key=pinecone_api_key)

In [17]:
pc

In [18]:
from pinecone import ServerlessSpec
index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )


index = pc.Index(index_name)


In [19]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=test_chunk,
    embedding=embeddings,
    index_name=index_name
)

In [22]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [25]:
#Adding documents to Pinecone
NEW_DOCS = Document(
    page_content="What is the capital of France?",
    metadata={"source": "France"}
)

docsearch.add_documents([NEW_DOCS])





['9018e958-cb8d-45f7-95d5-a80149641696']

In [27]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [30]:
retrieved_docs = retriever.invoke("what is Acne?")
retrieved_docs

[Document(id='a19b000f-76b5-4813-9fa7-c2a3fc6da263', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='a9df6119-3a46-4c70-b94a-a465e327d990', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='434fddd6-ee0d-4d9a-b66a-68de1d803c6a', metadata={'source': 'data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged wi

In [31]:
from langchain_openai import ChatOpenAI
chatModel = ChatOpenAI(model="gpt-4o-mini", temperature=0)












In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


In [39]:
system_prompt = """You are the Medical Assistant for Question and answering tasks.
Use the following pieces of retrieved context to answer the user's question.

Context: {context}

Remember: You are NOT a doctor and cannot provide medical diagnosis or treatment recommendations.
Always encourage users to consult healthcare professionals for personal medical concerns."""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])






In [40]:
question_answer_chain =create_stuff_documents_chain(chatModel,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [41]:
response=rag_chain.invoke({"input":"what is Acromegaly and gigantism?"})
print(response["answer"])

MaxRetryError: HTTPSConnectionPool(host='medical-chatbot-8zvi52e.svc.aped-4627-b74a.pinecone.io', port=443): Max retries exceeded with url: /query (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001DEBCE20A50>: Failed to resolve 'medical-chatbot-8zvi52e.svc.aped-4627-b74a.pinecone.io' ([Errno 11001] getaddrinfo failed)"))